<a href="https://colab.research.google.com/github/vithika-karan/Zomato-Restaurant-Clustering-and-Sentiment-Analysis/blob/main/Zomato_Restaurant_Clustering_and_Sentiment_Analysis_Vithika_Karan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem Statement**

Zomato is an Indian restaurant aggregator and food delivery start-up founded by Deepinder Goyal and Pankaj Chaddah in 2008. Zomato provides information, menus and user-reviews of restaurants, and also has food delivery options from partner restaurants in select cities.

India is quite famous for its diverse multi cuisine available in a large number of restaurants and hotel resorts, which is reminiscent of unity in diversity. Restaurant business in India is always evolving. More Indians are warming up to the idea of eating restaurant food whether by dining outside or getting food delivered. The growing number of restaurants in every state of India has been a motivation to inspect the data to get some insights, interesting facts and figures about the Indian food industry in each city. So, this project focuses on analysing the Zomato restaurant data for each city in India.

The Project focuses on Customers and Company, you have to analyze the sentiments of the reviews given by the customer in the data and made some useful conclusion in the form of Visualizations. Also, cluster the zomato restaurants into different segments. The data is vizualized as it becomes easy to analyse data at instant. The Analysis also solves some of the business cases that can directly help the customers finding the Best restaurant in their locality and for the company to grow up and work on the fields they are currently lagging in.

This could help in clustering the restaurants into segments. Also the data has valuable information around cuisine and costing which can be used in cost vs. benefit analysis

Data could be used for sentiment analysis. Also the metadata of reviewers can be used for identifying the critics in the industry. 

# **Attribute Information**

## **Zomato Restaurant names and Metadata**
Use this dataset for clustering part

1. Name : Name of Restaurants

2. Links : URL Links of Restaurants

3. Cost : Per person estimated Cost of dining

4. Collection : Tagging of Restaurants w.r.t. Zomato categories

5. Cuisines : Cuisines served by Restaurants

6. Timings : Restaurant Timings

## **Zomato Restaurant reviews**
Merge this dataset with Names and Matadata and then use for sentiment analysis part

1. Restaurant : Name of the Restaurant

2. Reviewer : Name of the Reviewer

3. Review : Review Text

4. Rating : Rating Provided by Reviewer

5. MetaData : Reviewer Metadata - No. of Reviews and followers

6. Time: Date and Time of Review

7. Pictures : No. of pictures posted with review

###**Notebook Breakdown:**
* Business Problem Analysis
* Data Collection
* Data Cleaning and Preprocessing
* Feature Engineering
* Exploratory Data Analysis
    
    - Hypotheses Generation
    - Visualizing Data
    - EDA Conclusion and Validation 
    - Hypoheses Generation for Cluster Analysis
* Restaurant Clustering
  - Best restaurants in different cities
* Sentiment Analysis


###**Business Problem Analysis**

###**Data Collection**

In [173]:
#Importing important libraries and modules
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.rcParams.update({'figure.figsize':(8,5),'figure.dpi':100})
from datetime import datetime

import warnings    
warnings.filterwarnings('ignore')

In [174]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [175]:
#reading datasets
rest_df = pd.read_csv("/content/drive/MyDrive/ALMABETTER/CAPSTONE PROJECTS/Zomato Restaurant Clustering and Sentiment Analysis - Vithika Karan /Data & Resources/Zomato Restaurant names and Metadata.csv")
reviews_df = pd.read_csv("/content/drive/MyDrive/ALMABETTER/CAPSTONE PROJECTS/Zomato Restaurant Clustering and Sentiment Analysis - Vithika Karan /Data & Resources/Zomato Restaurant reviews.csv")

In [176]:
# first five rows of restaurants dataset
rest_df.head()

,Name,Links,Cost,Collections,Cuisines,Timings
0,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)"
1,Paradise,https://www.zomato.com/hyderabad/paradise-gach...,800,Hyderabad's Hottest,"Biryani, North Indian, Chinese",11 AM to 11 PM
2,Flechazo,https://www.zomato.com/hyderabad/flechazo-gach...,"1,300","Great Buffets, Hyderabad's Hottest","Asian, Mediterranean, North Indian, Desserts","11:30 AM to 4:30 PM, 6:30 PM to 11 PM"
3,Shah Ghouse Hotel & Restaurant,https://www.zomato.com/hyderabad/shah-ghouse-h...,800,Late Night Restaurants,"Biryani, North Indian, Chinese, Seafood, Bever...",12 Noon to 2 AM
4,Over The Moon Brew Company,https://www.zomato.com/hyderabad/over-the-moon...,"1,200","Best Bars & Pubs, Food Hygiene Rated Restauran...","Asian, Continental, North Indian, Chinese, Med...","12noon to 11pm (Mon, Tue, Wed, Thu, Sun), 12no..."


In [177]:
#first five rows of reviews dataset
reviews_df.head()

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0


###**Data Cleaning and Preprocessing**

In [178]:
#restaurnts info - null count and dtypes 
rest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         105 non-null    object
 1   Links        105 non-null    object
 2   Cost         105 non-null    object
 3   Collections  51 non-null     object
 4   Cuisines     105 non-null    object
 5   Timings      104 non-null    object
dtypes: object(6)
memory usage: 5.0+ KB


Around 50% of the data is missing in the categorical column "Collections", which are basically just tags given by zomato for better search results.
Even when imputed with various categorical data imputing measures, it would be pretty difficult to match similar tags as the restaurants and then even more difficult to then convert them into a meaningful numerical feature afterward.
If the information contained in the variable is not that high, it is better to drop the variable if it has 50% or more missing values.

In [179]:
#drop collections
rest_df.drop('Collections', axis=1, inplace=True)

In [180]:
#Impute one missing timing row with the mode
rest_df['Timings'].fillna(rest_df['Timings'].mode()[0],inplace=True)

In [181]:
#check nulls
rest_df.isnull().sum()

Name        0
Links       0
Cost        0
Cuisines    0
Timings     0
dtype: int64

In [182]:
#reviews info - null count and dtypes 
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Restaurant  10000 non-null  object
 1   Reviewer    9962 non-null   object
 2   Review      9955 non-null   object
 3   Rating      9962 non-null   object
 4   Metadata    9962 non-null   object
 5   Time        9962 non-null   object
 6   Pictures    10000 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 547.0+ KB


In [183]:
#exploring null rows in reviews column
reviews_df[reviews_df['Review'].isnull()]

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures
2360,Amul,Lakshmi Narayana,NaN,5,0 Reviews,7/29/2018 18:00,0
5799,Being Hungry,Surya,NaN,5,"4 Reviews , 4 Followers",7/19/2018 23:55,0
6449,Hyderabad Chefs,Madhurimanne97,NaN,5,1 Review,7/23/2018 16:29,0
6489,Hyderabad Chefs,Harsha,NaN,5,1 Review,7/8/2018 21:19,0
7954,Olive Garden,ARUGULLA PRAVEEN KUMAR,NaN,3,"1 Review , 1 Follower",8/9/2018 23:25,0
8228,Al Saba Restaurant,Suresh,NaN,5,1 Review,7/20/2018 22:42,0
8777,American Wild Wings,NaN,NaN,NaN,NaN,NaN,0
8778,American Wild Wings,NaN,NaN,NaN,NaN,NaN,0
8779,American Wild Wings,NaN,NaN,NaN,NaN,NaN,0
8780,American Wild Wings,NaN,NaN,NaN,NaN,NaN,0


The "Review" column has text that needs to be analyzed to understand the sentiments and without it, the analysis cannot be done. It can also be seen that most of the null values in the review column also have nulls in other corresponding columns such as Reviewer, Rating, Metadata, and Time. These instances should be dropped.

In [184]:
#dropping null rows in reviews first
reviews_df.dropna(subset = ["Review"], inplace=True)

In [185]:
# checking
reviews_df.isnull().sum()

Restaurant    0
Reviewer      0
Review        0
Rating        0
Metadata      0
Time          0
Pictures      0
dtype: int64

###**Feature Engineering**

Feature engineering is the process of selecting, manipulating, and transforming raw data into meaningful numerical features that can be used by machine learning algorithms. 

First, the restaurants dataset has columns such as Links, Cuisine, and Timings which aren't directly interpretable.
The location of the restaurant can be extracted by the Links column.
Cuisines can be clubbed and categorized into a few categories and a total number of cuisines served by a particular restaurant.
Timings can be categorized into three categories to make analysis a little simpler.

**Links**

In [186]:
# link value
rest_df.loc[0,'Links']

'https://www.zomato.com/hyderabad/beyond-flavours-gachibowli'

In [187]:
#function to extract location of the restaurant
def location(link):
  link_elements = link.split("/")
  return link_elements[3]

#create a location feature
rest_df['Location'] = rest_df['Links'].apply(location)

In [188]:
# looks like the dataset consists of the restaurants in Hyderabad
rest_df['Location'].unique()

array(['hyderabad', 'thetiltbarrepublic'], dtype=object)

In [189]:
# exploring the other value
rest_df[rest_df.isin(['thetiltbarrepublic'])].stack()

68  Location    thetiltbarrepublic
dtype: object

In [190]:
#doesnt have location
rest_df.loc[68,:]

Name                            The Tilt Bar Republic
Links       https://www.zomato.com/thetiltbarrepublic
Cost                                            1,500
Cuisines           North Indian, Continental, Italian
Timings                12noon to 12midnight (Mon-Sun)
Location                           thetiltbarrepublic
Name: 68, dtype: object

In [191]:
#dropping unnecessary columns
rest_df.drop(['Links','Location'],axis=1,inplace=True)

**Cuisines**

In [192]:
pd.set_option('display.max_rows', 100)

In [193]:
#splitting to create list instead of strings
rest_df['Cuisines'] = rest_df['Cuisines'].apply(lambda x : x.split(','))

#creating a list of all cuisine lists for different restaurants
cuisine_list = []
for idx in rest_df.index:
  cuisine_list.append(rest_df['Cuisines'][idx])

#creating a flat list
cuisine_list = [item for sublist in cuisine_list for item in sublist]

In [169]:
#frequency dict
frequency_dict = {}
for elem in cuisine_list:
  if elem not in frequency_dict.keys():
    frequency_dict[elem] = cuisine_list.count(elem)
  else:
    pass

#frequency table
cuisine_df = pd.DataFrame(list(frequency_dict.items()),columns=['Cuisine','Frequency'])
cuisine_df = cuisine_df.sort_values(by = "Frequency", ascending = False,ignore_index=True)
cuisine_df.head(70)

,Cuisine,Frequency
0,Chinese,36
1,North Indian,33
2,North Indian,28
3,Continental,17
4,Biryani,12
5,Italian,12
6,Desserts,11
7,Fast Food,10
8,Asian,10
9,Chinese,7


In [194]:
#unique list of cuisines
unique_cuisines = list(set(cuisine_list))

In [195]:
#minimising the number of cuisines by sorting and categorizing them out
cuisine_dict = {'Chinese':['Chinese',' Chinese'],'North Indian':['North India',' North Indian'],'Continental':['Continental',' Continental']
                }


{' American': 2,
 ' Andhra': 3,
 ' Arabian': 1,
 ' Asian': 10,
 ' BBQ': 1,
 ' Bakery': 1,
 ' Beverages': 5,
 ' Biryani': 12,
 ' Burger': 3,
 ' Cafe': 1,
 ' Chinese': 36,
 ' Continental': 17,
 ' Desserts': 11,
 ' European': 2,
 ' Fast Food': 10,
 ' Finger Food': 1,
 ' Goan': 1,
 ' Hyderabadi': 3,
 ' Indonesian': 1,
 ' Italian': 12,
 ' Japanese': 2,
 ' Juices': 1,
 ' Kebab': 5,
 ' Malaysian': 1,
 ' Mediterranean': 4,
 ' Mithai': 1,
 ' Modern Indian': 1,
 ' Momos': 3,
 ' Mughlai': 5,
 ' North Indian': 28,
 ' Pizza': 1,
 ' Salad': 5,
 ' Seafood': 3,
 ' South Indian': 7,
 ' Spanish': 1,
 ' Sushi': 4,
 ' Thai': 2,
 ' Wraps': 1,
 'American': 4,
 'Andhra': 3,
 'Arabian': 1,
 'Asian': 5,
 'BBQ': 1,
 'Bakery': 6,
 'Biryani': 4,
 'Burger': 2,
 'Cafe': 5,
 'Chinese': 7,
 'Continental': 4,
 'Desserts': 2,
 'European': 2,
 'Fast Food': 5,
 'Finger Food': 1,
 'Healthy Food': 1,
 'Hyderabadi': 1,
 'Ice Cream': 2,
 'Italian': 2,
 'Kebab': 1,
 'Lebanese': 1,
 'Mediterranean': 1,
 'Mexican': 1,
 'Modern 

In [171]:
#explore unique cuisines, that's a lot
len(unique_cuisines)

70